In [1]:
# Install dependencies
!pip install flask pyngrok -q
!pip install transformers==4.46.2 accelerate==1.1.1 bitsandbytes==0.45.2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.5 MB/s eta 0:00:00


In [ ]:
# Auth Token Generated on https://dashboard.ngrok.com/get-started/your-authtoken
!ngrok config add-authtoken <redacted>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
# --- Step 3: Import libs ---
import torch
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# --- Load Model ---
model_name = "TejasKoti/Llama-2-7b-lawyer-chat"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    do_sample=True,
    top_p=0.9,
    temperature=0.7
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [10]:
# --- Step 5: Flask API (Improved & Longer Outputs) ---
from flask import Flask, request, jsonify
from pyngrok import ngrok
import torch

app = Flask(__name__)

# If you have access to tokenizer, set these once (optional but ideal)
try:
    EOS_ID = getattr(pipe.tokenizer, "eos_token_id", None)
    PAD_ID = getattr(pipe.tokenizer, "pad_token_id", 0)
except Exception:
    EOS_ID, PAD_ID = None, 0

@app.route("/generate", methods=["POST"])
def generate():
    data = request.get_json(silent=True) or {}
    question = (data.get("question") or "").strip()
    if not question:
        return jsonify({"error": "No question provided"}), 400

    # 🧠 System-style instruction to enforce structure & clarity
    sys_prompt = (
        "You are LawyerAI, a knowledgeable, polite legal assistant. "
        "Answer comprehensively with clear structure. Use this template:\n"
        "## Summary (2–3 lines)\n"
        "## Key Steps (numbered)\n"
        "## Important Considerations (bullets)\n"
        "## Next Actions (bullets)\n"
        "Avoid legal jargon unless needed; be concise and practical."
    )

    # For Llama/Mistral-style instruct models this works well
    prompt = (
        f"<s>[INST] {sys_prompt}\n\nUser question: {question} [/INST]"
    )

    # ⚙️ Generation parameters tuned for longer, cleaner outputs
    gen_kwargs = dict(
        do_sample=True,
        temperature=0.65,         # balanced clarity/creativity
        top_p=0.92,
        typical_p=0.95,           # (supported by many HF models)
        repetition_penalty=1.12,  # reduce loops
        no_repeat_ngram_size=3,   # avoid repeated phrases
        max_new_tokens=900,       # 🔥 allow long answers
        min_new_tokens=120,       # ensure a minimum size
        eos_token_id=EOS_ID if EOS_ID is not None else (2 if "llama" in str(type(getattr(pipe, "model", ""))).lower() else None),
        pad_token_id=PAD_ID,
        # If your pipeline supports it, this avoids echoing the prompt:
        return_full_text=False
    )

    with torch.inference_mode():
        outputs = pipe(prompt, **gen_kwargs)

    # Some pipelines return dicts; some return strings
    generated = outputs[0].get("generated_text", outputs[0])

    # 🧹 Clean the model output
    if "[/INST]" in generated:
        response = generated.split("[/INST]", 1)[-1].strip()
    else:
        response = generated.strip()

    for bad in ("<s>", "</s>", "[INST]", "</INST>"):
        response = response.replace(bad, "")
    response = response.strip()

    # Optional: normalize excessive blank lines
    while "\n\n\n" in response:
        response = response.replace("\n\n\n", "\n\n")

    # Return plain text (frontend should render \n as <br> or Markdown)
    return jsonify({"response": response})


# --- Step 6: Expose via ngrok ---
public_url = ngrok.connect(5000)
print("🔗 Public API URL:", public_url)
app.run(port=5000)

🔗 Public API URL: NgrokTunnel: "https://beholden-markus-nonsuggestively.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Both `max_new_tokens` (=900) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
INFO:werkzeug:127.0.0.1 - - [17/Oct/2025 10:27:21] "POST /generate HTTP/1.1" 200 -
